# Pauli Spin Blockade

Blah blah intro text.

## Intro to Spaghetti Diagrams

Recall the [stability diagrams](sec:stability-diagrams) of double-dots from earlier. Suppose we want to examine the cases where there are exactly two electrons in the double-dot. In particular, we could examine a diagonal slice of the stablility diagram that passes through the (2,0), (1,1), and (0,2) regions. Movement along this line can be characterized in terms of the **bias** $\epsilon=\mu_2-\mu_1$---the difference in potentials of the two dots. Clearly, when the bias is high, the (2,0) state is more likely; when the bias is very negative, the (0,2) state is most favorable, and when the bias is small in magnitude the (1,1) state is most common. But we'd like to be more quantitative than that. We can create a simple model for this system, and then build on it later.

We will assume the tunnel coupling between the dots is a constant $t_c$, and that having two electrons in the same dot increases the energy of the system by a constant $U$ due to Coulomb repulsion. The energies of the (2,0), (1,1), and (0,2) states are thus $2\mu_1+U$, $\mu_1+\mu_2$, and $2\mu_2+U$ respectively. To simplify the math, let us instead set $\frac{1}{2}(\mu_1+\mu_2)$ as our new 0 for energy, so that $\mu_1$ becomes $-\frac{\epsilon}{2}$ and $\mu_2$ becomes $\frac{\epsilon}{2}$. Then the energies can be expressed as $U-\epsilon$, 0, and $U+\epsilon$. To construct a Hamiltonian, all we need are chances of moving from state to state. From the (1,1) state, you can move to either of the other states with chance $t_c$---one electron tunnels to the other dot to join its counterpart. From the (2,0) state, one electron can tunnel to the other dot to form the (1,1) state with chance $t_c$. However, it's also possible for the *other* electron to do so instead. In any case, there is no way to move immediately to the (0,2) case. Putting this all together, we get $\begin{equation}\hat H=\begin{pmatrix}U-\epsilon&2t_c&0\\t_c&0&t_c\\0&2t_c&U+\epsilon\end{pmatrix}.\end{equation}$

Solving for the eigenvalues is technically analytically possible, but not particularly illuminating. Instead let's see a graph of them for various values of $\epsilon$.

In [35]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, MultiLine
from bokeh.plotting import figure, show, save, output_file
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html
import IPython
UMAX = 10
tc = .3
YMIN = -2
YMAX = 4
NUM = 100
biases = np.linspace(-2 * UMAX, 2 * UMAX, NUM)
lambda1 = np.zeros(NUM)
lambda2 = np.zeros(NUM)
lambda3 = np.zeros(NUM)

def determinant(epsilon, U=UMAX, tc=tc):
    """Calculate determinant of H - lambda I for given bias epsilon."""
    return np.roots(np.array([1, -2 * U, U**2 - epsilon**2 - 4 * tc**2, 4 * U * tc**2]))

roots = np.zeros((NUM, 3))
for i in range(NUM):
    roots[i] = determinant(biases[i])
# there is only one crossing, of the (2,0)&(0,2) states at \epsilon=0.
# it's probably out of graph range, but...

sorted = np.sort(roots)
halfway = (NUM-1) // 2 + 1
# (2,0)
lambda1[:halfway] = sorted[:halfway,1]
lambda1[halfway:] = sorted[halfway:,2]
# (1,1)
lambda2 = sorted[:,0]
# (0,2)
lambda3[:halfway] = sorted[:halfway,2]
lambda3[halfway:] = sorted[halfway:,1]

source = ColumnDataSource(data=dict(biases=biases, lambda1=lambda1, lambda2=lambda2, lambda3=lambda3))

plot = figure(y_range=(YMIN, YMAX), x_range=(-2 * UMAX, 2 * UMAX), width=400, height=400,
              x_axis_label=r"$$\text{bias }\epsilon\text{ (energy)}$$",
              y_axis_label=r"$$E\text{ Energy}$$")

plot.line('biases', 'lambda1', source=source)
plot.line('biases', 'lambda2', source=source)
plot.line('biases', 'lambda3', source=source)

U_slider = Slider(start=1, end=1.5 * UMAX, value=UMAX, step=.05, title=r"$$U$$")
tc_slider = Slider(start=0.05, end=0.5, value=tc, step=.01, title=r"$$t_c$$")

callback = CustomJS(args=dict(source=source, Uslider=U_slider, tcslider=tc_slider, num=NUM),
                    code="""
    const U = Uslider.value;
    const tc = tcslider.value;
    const NUM = num;
    const HALFWAY = Math.floor((NUM - 1) / 2) + 1;

    const biases = source.data.biases;
    const lambda1 = [];
    const lambda2 = [];
    const lambda3 = [];

    var roots = [];
    for (let i = 0; i < biases.length; i++) {
        roots = math.polynomialRoot(2 * U * tc * tc, U * U - biases[i] * biases[i] - 2 * tc * tc, -2 * U, 1);
        roots.sort();
        lambda2.push(roots[0]);
        if (i < HALFWAY) {
            lambda1.push(roots[1]);
            lambda3.push(roots[2]);
        } else {
            lambda3.push(roots[1]);
            lambda1.push(roots[2]);
        }
    }

    source.data = { biases, lambda1, lambda2, lambda3 }
""")

U_slider.js_on_change('value', callback)
tc_slider.js_on_change('value', callback)

html_repr = file_html(row(plot, column(U_slider, tc_slider)), CDN)
IPython.display.HTML(html_repr)
# debugging
# output_file(filename="/home/peter/tmp.html")
# save(plot)

'/home/peter/tmp.html'

This is a simple "spaghetti diagram," and it (sort of) tells us some things about the dynamics of the system. At each of the anticrossings at $\epsilon=\pm U$, the mostly likely state changes from (0,2) to (1,1) or (1,1) to (2,0). In most spagetti diagrams, you're going to see a whole lot of anticrossings (a.k.a. "avoided crossings"), so let's go over them in more detail. 

The energy of the ground state is graphed as the lowest curve, and, as the name "anticrossing" suggests, it never actually switches places with another eigenvalue. So where does the state change come from? Let's look back at the Hamiltonian. Let's assume $t_c\ll U\pm\epsilon$. Then the matrix is basically already diagonalized, and we can read the eigenvalues of the diagonal. When $|\epsilon|\ll U$, the ground state is at about 0 energy, and the eigenstate is basically (1,1). This is the middle portion of our graph. But when $|\epsilon|\gg U$, the ground state becomes $U\pm\epsilon$, and the eigenstate changes to either (2,0) or (0,2)! So, while the eigenvalues do not cross, the eigenstates do switch around, which is why the favored state changes. 

We can also look at the case where $\epsilon=U$. Looking at the upper left 2x2 block of the Hamiltonian, $\begin{pmatrix}0&2t_c\\t_c&0\end{pmatrix}$, we can see two of the eigenvalues will be $\lambda=\pm\sqrt{2}t_c$, with corresponding eigenstates $\frac{1}{\sqrt{3}}\begin{pmatrix}\pm\sqrt{2}\\1\\0\end{pmatrix}$. So the system is in a pretty even mix of the (2,0) and (1,1) states. as $\epsilon$ increases, eigenstate associated with the ground state approaches $\begin{pmatrix}1\\0\\0\end{pmatrix}$. 

## A New Spin on Things

Up until now we have ignored electron spin, but of course the spin states of the electrons affect the energy of having two electrons in the same dot. The spin singlet state has energy $E_S=U\pm\epsilon$. We will assume there are no magnetic fields for the moment, so the three spin-triplets states are degenerate with energy $E_T=E_S+J$. 

So, there are now six states our system can be in: still (2,0), (1,1), and (0,2), but now in each case the electrons can be in either singlet or triplet states. We will assume the electrons' spin does not change over the timescales we care about. So, the Hamiltonian can simply be written as $\hat H=\left(\begin{array}{c|c}\hat H_S&0\\\hline0&\hat H_T\end{array}\right)$, where $\hat H_S$ is the same as our Hamiltonian above and $\hat H_T=\begin{pmatrix}U+J-\epsilon&2t_c&0\\t_c&0&t_c\\0&2t_c&U+J+\epsilon\end{pmatrix}$. In other words, it looks exactly the same except with anticrossings at $\epsilon=\pm(U+J)$. The graph would thus look like this:

In [ ]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, MultiLine
from bokeh.plotting import figure, show, save, output_file
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html
import IPython
UMAX = 10
tc = .3
YMIN = -2
YMAX = 4
NUM = 100
biases = np.linspace(-2.5 * UMAX, 2.5 * UMAX, NUM)
lambda1 = np.zeros(NUM)
lambda2 = np.zeros(NUM)
lambda3 = np.zeros(NUM)
triplet1 = np.zeros(NUM)
triplet2 = np.zeros(NUM)
triplet3 = np.zeros(NUM)

def determinant(epsilon, U=UMAX, tc=tc):
    """Calculate determinant of H - lambda I for given bias epsilon."""
    return np.roots(np.array([1, -2 * U, U**2 - epsilon**2 - 4 * tc**2, 4 * U * tc**2]))

roots = np.zeros((NUM, 3))
for i in range(NUM):
    roots[i] = determinant(biases[i])
# there is only one crossing, of the (2,0)&(0,2) states at \epsilon=0.
# it's probably out of graph range, but...

sorted = np.sort(roots)
halfway = (NUM-1) // 2 + 1
# (2,0)
lambda1[:halfway] = sorted[:halfway,1]
lambda1[halfway:] = sorted[halfway:,2]
# (1,1)
lambda2 = sorted[:,0]
# (0,2)
lambda3[:halfway] = sorted[:halfway,2]
lambda3[halfway:] = sorted[halfway:,1]

for i in range(NUM):
    roots[i] = determinant(biases[i], U=UMAX*1.5)

sorted = np.sort(roots)
halfway = (NUM-1) // 2 + 1
# (2,0)
triplet1[:halfway] = sorted[:halfway,1]
triplet1[halfway:] = sorted[halfway:,2]
# (1,1)
triplet2 = sorted[:,0]
# (0,2)
triplet3[:halfway] = sorted[:halfway,2]
triplet3[halfway:] = sorted[halfway:,1]

source = ColumnDataSource(data=dict(biases=biases, lambda1=lambda1, lambda2=lambda2, lambda3=lambda3,
                                    triplet1=triplet1, triplet2=triplet2, triplet3=triplet3))

plot = figure(y_range=(YMIN, YMAX), x_range=(-2.5 * UMAX, 2.5 * UMAX), width=400, height=400,
              x_axis_label=r"$$\text{bias }\epsilon\text{ (energy)}$$",
              y_axis_label=r"$$E\text{ (energy)}$$")

plot.line('biases', 'lambda1', source=source, color='blue', legend_label="Singlet")
plot.line('biases', 'lambda2', source=source, color='blue')
plot.line('biases', 'lambda3', source=source, color='blue')

plot.line('biases', 'triplet1', source=source, color='red', legend_label="Triplet")
plot.line('biases', 'triplet2', source=source, color='red')
plot.line('biases', 'triplet3', source=source, color='red')

U_slider = Slider(start=1, end=1.5 * UMAX, value=UMAX, step=.05, title=r"$$U$$")
tc_slider = Slider(start=0.05, end=0.5, value=tc, step=.01, title=r"$$t_c$$")

# update if above code ever gets working
callback = CustomJS(args=dict(source=source, Uslider=U_slider, tcslider=tc_slider, num=NUM),
                    code="""
    const U = Uslider.value;
    const tc = tcslider.value;
    const NUM = num;
    const HALFWAY = Math.floor((NUM - 1) / 2) + 1;

    const biases = source.data.biases;
    const lambda1 = [];
    const lambda2 = [];
    const lambda3 = [];

    var roots = [];
    for (let i = 0; i < biases.length; i++) {
        roots = math.polynomialRoot(2 * U * tc * tc, U * U - biases[i] * biases[i] - 2 * tc * tc, -2 * U, 1);
        roots.sort();
        lambda2.push(roots[0]);
        if (i < HALFWAY) {
            lambda1.push(roots[1]);
            lambda3.push(roots[2]);
        } else {
            lambda3.push(roots[1]);
            lambda1.push(roots[2]);
        }
    }

    source.data = { biases, lambda1, lambda2, lambda3 }
""")

U_slider.js_on_change('value', callback)
tc_slider.js_on_change('value', callback)

html_repr = file_html(row(plot, column(U_slider, tc_slider)), CDN)
IPython.display.HTML(html_repr)
# debugging
# output_file(filename="/home/peter/tmp.html")
# save(plot)

Note that when $U<|\epsilon|<U+J$, the electrons will prefer a (0,2) or (2,0) state if they are in a spin-singlet state, but they will prefer a (1,1) state if they are a spin-triplet. This is the basis for Pauli spin blockade: Set up a source and drain on each side of a double dot containing two electrons in a (1,1) state with unknown spins, with $\mu_1(0)<\mu_\text{source}<\mu_1(1)$ and $\mu_2(1)<\mu_\text{drain}<\mu_2(2)$. Then, the bias is set to be $-U-J<\epsilon<-J$. If the electrons are in a spin-singlet state, the electron in dot 1 will be happy to hop to dot 2 and then from there into the drain. Another electron will load into dot 1 from the source and the process can repeat, creating a current. On the other hand, if the electrons are in a triplet state, the (0,2) state is not favorable, so the electron in dot 1 will not move. This creates a "blockade" and prevents current flow. Current is easier to measure than directly measuring spin, so using this phenomenon to determine spin states is very convenient. 